# Скачиваем полезное

In [1]:
! pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 46.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=fb4dd452c29a40d8f04c51a081ab3d5f1b774558c60e4b75dd33e33945d5ee95
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [3]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [4]:
import gc
import os
import csv
import glob
import math
from functools import reduce
from time import time

import random
from collections import Counter
from razdel import tokenize
from typing import Any, Dict, List, Optional, Tuple, Union
import unittest

import numpy as np
import pandas as pd
import torch
from torch.optim import AdamW
from torch import nn
import torch.nn.functional as F


# Портим текст

In [5]:
class TextIlliteracy:
    """Class for tokenizing text and changing variant of grammatical category
       in words of chosen parts of speech"""
    _tokens: List[str]

    def __init__(self, text: str) -> None:
        """Initializes an object"""
        self._text = text
        self._tokens = []
    
    def get_original_text(self) -> str:
        return self._text

    def tokenize_text(self) -> List[str]:
        """Tokenizes text"""
        pass

    def spoil_text(self, gram: str, postag_list: List[str]) -> str:
        """Changes grammatical markers to random inside the choosen category
           in all words of choosen parts of speech,
           if this grammatical category is relevant for such POS"""
        pass

In [ ]:
class TextIlliteracyRus(TextIlliteracy):
    """Class tokenize russian text and changes variant of grammatical category
       in words of chosen parts of speech."""
    
    _tokens: List[str]
    # https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs
    
    __postags = ["NOUN", "ADJF", "ADJS", "COMP", "VERB", "INFN", "PRTF", "PRTS",
                 "GRND", "NUMR", "ADVB", "NPRO", "PRED", "PREP", "CONJ", "PRCL", "INTJ"]
    
    __grams = {"nmbr": ["sing", "plur"],
               "case": ["nomn", "gent", "datv", "accs", "ablt",
                        "loct", "voct", "gen2", "acc2", "loc2"],
               "anim": ["anim", "inan"],
               "gndr": ["masc", "femn", "neut", "ms-f"],
               "aspc": ["perf", "impf"],
               "trns": ["tran", "intr"],
               "pers": ["1per", "2per", "3per"],
               "tens": ["pres", "past", "futr"],
               "mood": ["indc", "impr"],
               "invl": ["incl", "excl"],
               "voic": ["actv", "pssv"]}

    def tokenize_text(self) -> List[str]:
        """For russian:
           tokenizes text"""
        if self._tokens != []:
            tokens = self._tokens
        else:
            tokens_with_boundaries = list(tokenize(self._text))
            # получили список токенов с границами
            tokens = [] # список токенов с пробелами в нужных местах
            prev_tok_end = 0
            for substring in tokens_with_boundaries:
                if substring.start != prev_tok_end:
                    tokens.append(" ")
                tokens.append(substring.text)
                prev_tok_end = substring.stop
            self._tokens = tokens
        return tokens
        
    def spoil_text(self, gram: str="nmbr", postag_list: List[str]=__postags) -> str:
        """For russian:
           changes grammatical markers to random inside the choosen category
           in all words of choosen parts of speech,
           if this grammatical category is relevant for such POS"""
        # берёт список частей речи и категорию,
        # которую у этих частей речи надо портить рандомными вариантами
        
        if self._tokens == []:
            self._tokens = self.tokenize_text()

        tokens = self._tokens

        changed_list = []
        for tok in tokens:
            tok_analysed = morph.parse(tok)[0]
            if tok_analysed.tag.POS in postag_list:
                new_gram_val = random.choice(TextIlliteracyRus.__grams[gram])
                
                if new_gram_val in tok_analysed.tag:  # делает вероятнось повторения формы чуть меньше
                    new_gram_val = random.choice(TextIlliteracyRus.__grams[gram])

                if tok_analysed.inflect({new_gram_val}) is not None:
                    changed_tok = tok_analysed.inflect({new_gram_val}).word
                    if tok[0].isupper():
                        # чтобы при изменении буква оставалась заглавной в т.ч. для слова "Ма́трица"
                        changed_tok = changed_tok[0].upper()+changed_tok[1:]
                    changed_list.append(changed_tok)
                else:
                    changed_list.append(tok)
            else:
                changed_list.append(tok)
        changed_text = "".join(changed_list)
        return changed_text

In [ ]:
from tqdm import tqdm

In [ ]:
# для других языков порча текста может основываться на других специализированных библиотеках
# в том числе на трансдьюсерах
# class TextIlliteracyEng, например

## Примеры работы класса

In [ ]:
text = """Ма́трица — математический объект, записываемый в виде прямоугольной таблицы элементов кольца или поля (например, целых, действительных или комплексных чисел), который представляет собой совокупность строк и столбцов, на пересечении которых находятся его элементы. Количество строк и столбцов задает размер матрицы. Хотя исторически рассматривались, например, треугольные матрицы[1], в настоящее время говорят исключительно о матрицах прямоугольной формы, так как они являются наиболее удобными и общими.
Матрицы широко применяются в математике для компактной записи систем линейных алгебраических или дифференциальных уравнений. В этом случае количество строк матрицы соответствует числу уравнений, а количество столбцов — количеству неизвестных. В результате решение систем линейных уравнений сводится к операциям над матрицами."""
a = TextIlliteracyRus(text)
a.get_original_text()
a.tokenize_text()
#a.spoil_text("case", ["NOUN", "ADJF"])
a.spoil_text("nmbr", ["NOUN"])

'Ма́трица — математический объекты, записываемый в видах прямоугольной таблиц элемента колец или полей (например, целых, действительных или комплексных чисел), который представляет собой совокупности строки и столбцов, на пересечениях которых находятся его элемент. Количество строки и столбцов задает размеры матриц. Хотя исторически рассматривались, например, треугольные матриц[1], в настоящее времена говорят исключительно о матрицах прямоугольной формы, так как они являются наиболее удобными и общими. Матриц широко применяются в математикам для компактной записей систем линейных алгебраических или дифференциальных уравнения. В этом случаях количества строк матриц соответствует числу уравнения, а количества столбцов — количествам неизвестных. В результатах решение системы линейных уравнений сводится к операциям над матрицей.'

Текст от DeepPavlov

In [ ]:
spoiled_lines = []
filename = '/content/drive/MyDrive/ВШЭ/Магистратура/Perplexity_probing/preprocessed_ru_convers.txt'
with open(filename, encoding='UTF-8') as f:
    for i, line in tqdm(enumerate(f)):
        a = TextIlliteracyRus(line)
        spoiled_line = a.spoil_text("gndr", ["ADJF"])
        spoiled_lines.append(spoiled_line)

3092it [00:13, 235.66it/s]


KeyboardInterrupt: ignored

In [ ]:
spoiled_lines[:5]

['А вообще уже было Здесь ещё точно не было!',
 'Я понял — это такой хитрая план.',
 'Перетащить сюда все золотое посты со старого d3 А я что–то пропустил то пост.',
 'Ну, да ладно.',
 'Я сделал пост для проверки.']

In [ ]:
filename_out = 'preprocessed_ru_convers_spoiled_ADJF_gnrd.txt'
with open(filename_out, 'w', encoding='UTF-8') as output:
    for line in spoiled_lines:
        output.write(line)
        output.write('\n')

In [ ]:
# Если хотим портить другое,
# то меняем строку "spoiled_line = a.spoil_text("gndr", ["ADJF"])"
# spoiled_line = a.spoil_text("gndr", ["VERB"]) - род в глаголах
# spoiled_line = a.spoil_text("nmbr", ["NOUN"]) - число в существительных


Для 100% порчи:

In [30]:
class TextIlliteracyRus(TextIlliteracy):
    """Class tokenize russian text and changes variant of grammatical category
       in words of chosen parts of speech."""
    
    _tokens: List[str]
    # https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs
    
    __postags = ["NOUN", "ADJF", "ADJS", "COMP", "VERB", "INFN", "PRTF", "PRTS",
                 "GRND", "NUMR", "ADVB", "NPRO", "PRED", "PREP", "CONJ", "PRCL", "INTJ"]
    
    __grams = {"nmbr": ["sing", "plur"],
               "case": ["nomn", "gent", "datv", "accs", "ablt",
                        "loct", "voct", "gen2", "acc2", "loc2"],
               "anim": ["anim", "inan"],
               "gndr": ["masc", "femn", "neut", "ms-f"],
               "aspc": ["perf", "impf"],
               "trns": ["tran", "intr"],
               "pers": ["1per", "2per", "3per"],
               "tens": ["pres", "past", "futr"],
               "mood": ["indc", "impr"],
               "invl": ["incl", "excl"],
               "voic": ["actv", "pssv"]}

    def tokenize_text(self) -> List[str]:
        """For russian:
           tokenizes text"""
        if self._tokens != []:
            tokens = self._tokens
        else:
            tokens_with_boundaries = list(tokenize(self._text))
            # получили список токенов с границами
            tokens = [] # список токенов с пробелами в нужных местах
            prev_tok_end = 0
            for substring in tokens_with_boundaries:
                if substring.start != prev_tok_end:
                    tokens.append(" ")
                tokens.append(substring.text)
                prev_tok_end = substring.stop
            self._tokens = tokens
        return tokens
        
    def spoil_text(self, gram: str="nmbr", postag_list: List[str]=__postags) -> str:
        """For russian:
           changes grammatical markers to random inside the choosen category
           in all words of choosen parts of speech,
           if this grammatical category is relevant for such POS"""
        # берёт список частей речи и категорию,
        # которую у этих частей речи надо портить рандомными вариантами
        
        if self._tokens == []:
            self._tokens = self.tokenize_text()

        tokens = self._tokens

        changed_list = []
        for tok in tokens:
            tok_analysed = morph.parse(tok)[0]
            if tok_analysed.tag.POS in postag_list:

                word_grams = str(tok_analysed.tag).split()[1].split(',')  # убирает вариант, в котором сейчас слово
                variants = TextIlliteracyRus.__grams[gram].copy()
                for i in range(len(word_grams)):
                    if word_grams[i] in variants:
                        variants.remove(word_grams[i])

                new_gram_val = random.choice(variants)
                if tok_analysed.inflect({new_gram_val}) is not None:
                    changed_tok = tok_analysed.inflect({new_gram_val}).word
                    if tok[0].isupper():
                        # чтобы при изменении буква оставалась заглавной в т.ч. для слова "Ма́трица"
                        changed_tok = changed_tok.capitalize()
                    changed_list.append(changed_tok)
                else:
                    changed_list.append(tok)
            else:
                changed_list.append(tok)
        changed_text = "".join(changed_list)
        return changed_text

In [31]:
text = """Ма́трица — математический объект, записываемый в виде прямоугольной таблицы элементов кольца или поля (например, целых, действительных или комплексных чисел), который представляет собой совокупность строк и столбцов, на пересечении которых находятся его элементы. Количество строк и столбцов задает размер матрицы. Хотя исторически рассматривались, например, треугольные матрицы[1], в настоящее время говорят исключительно о матрицах прямоугольной формы, так как они являются наиболее удобными и общими.
Матрицы широко применяются в математике для компактной записи систем линейных алгебраических или дифференциальных уравнений. В этом случае количество строк матрицы соответствует числу уравнений, а количество столбцов — количеству неизвестных. В результате решение систем линейных уравнений сводится к операциям над матрицами."""
a = TextIlliteracyRus(text)
a.get_original_text()
a.tokenize_text()
#a.spoil_text("case", ["NOUN", "ADJF"])
a.spoil_text("nmbr", ["NOUN"])

'Ма́трица — математический объекты, записываемый в видах прямоугольной таблиц элемента колец или полей (например, целых, действительных или комплексных числа), который представляет собой совокупности строки и столбцов, на пересечениях которых находятся его элемент. Количества строки и столбцов задает размеры матриц. Хотя исторически рассматривались, например, треугольные матриц[1], в настоящее времена говорят исключительно о матрице прямоугольной форм, так как они являются наиболее удобными и общими. Матриц широко применяются в математикам для компактной записей системы линейных алгебраических или дифференциальных уравнения. В этом случаях количества строки матриц соответствует числам уравнения, а количества столбцов — количествам неизвестных. В результатах решения системы линейных уравнения сводится к операции над матрицей.'